In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import subprocess

try:
    from google.cloud import drive
    drive.mount('/content/drive')

    os.chdir('/content/drive/Shareddrives/ba_team/_prod/mst_doanh_nghiep_v2')
    subprocess.run(['pip','install','-r','setup/requirements.txt'])
except:
    pass

import telegram
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd

from src.lib.etl.class_ETL import ETL

load_dotenv('./src/bin/.env')
bot_tele = telegram.Bot(token= os.getenv('BOT_TOKEN'))

In [2]:
# # This data was created by using the curl method explained above
# headers_list = [
#     {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",},
#     {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:76.0) Gecko/20100101 Firefox/76.0",},
#     {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1",},
#     {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"},
#     {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',},
# ]

# # Replace with your authenticated proxy URL
# proxyList = []

# df = pd.read_csv(rf'./../../_config/iproyal-proxies (1).csv')

# for idx, row in df.iterrows():
#     proxy = 'http://' + row['User'] + ':' + row['Pass'] +'@'+ row['Host'] + ':' + str(row['Port'])
#     proxyList.append(proxy)


In [2]:
triendt = ETL()

bigquery_data = triendt.fetch_data_from_bigquery(
    _file_path='./../../_config/cred.json',
    _update_time= '2024-04-01',                 ####   <<------ Chọn thời gian mst được thêm mới/ cập nhật. Và phải UNCOMMENT dòng này. Nếu không thì mặc định thời gian check từ 30 trờ về trước bắt đầu từ hôm nay
    )

data_local = triendt.fetch_data_local_db(db_path='./data/local_db/database.db')

data_to_validate = set(bigquery_data.tax_code)
data_to_validate.difference_update(set(data_local.tax_code))

In [4]:
await bot_tele.send_message(
            chat_id= os.getenv('GROUP_TEST_ID'),
            text= f'Start checking at {str(datetime.now())}'
        )

file_info = {
    'dir' : f'./data/process/{str(datetime.now().date())}',
    'file_name' : f'data_run_at_{str(datetime.now().hour)}.csv',
    'relative_file_path': f'./data/process/{str(datetime.now().date())}' + '/' +  f'data_run_at_{str(datetime.now().hour)}.csv'
}

for i, ele in enumerate(data_to_validate):
    if not ele.strip():
        break

    data = triendt.transform_data_from_api(_tax_code = ele.strip())
    
    if data['name'] == 'No data':
        data = triendt.transform_data_from_web(_tax_code=ele.strip())
        
    if data['name'] != 'No data':
        result = triendt.update_data_local_db(db_path= './data/local_db/database.db',
                                     _data_to_update= (data['id'], data['name'], datetime.now().date())
                                     )
    
    else:
        user_id = bigquery_data[bigquery_data.tax_code.str.contains(ele.strip())]['user_id'].drop_duplicates().tolist()[0]
        result = triendt.write_data_to_csv_file(
                    _data= f"{str(ele.strip())},No data,No data,{user_id}",
                    file_name = file_info['file_name'], 
                    _file_path= file_info['dir']
                )


await bot_tele.send_message(
            chat_id= os.getenv('GROUP_TEST_ID'),
            text= f'Finished at {str(datetime.now())}'
        )

await bot_tele.send_document(
            chat_id= os.getenv('GROUP_TEST_ID'),
            document = open( file_info['relative_file_path'], 'rb'), 
            caption = 'List of customers that provided unvalid tax code'
        )


Message(caption='List of customers that provided unvalid tax code', channel_chat_created=False, chat=Chat(id=-1002118260026, title='[FA - BA] Test Bot', type=<ChatType.SUPERGROUP>, username='ahm_fa_test'), date=datetime.datetime(2024, 5, 5, 9, 36, 35, tzinfo=<UTC>), delete_chat_photo=False, document=Document(file_id='BQACAgUAAx0EfkIVOgADZmY3UyPeJuMaOLxvmTe_3q0HmlIhAAI9EgACKGrBVTQ_Bj-Zk-bKNQQ', file_name='data_run_at_16.csv', file_size=371, file_unique_id='AgADPRIAAihqwVU', mime_type='text/csv'), from_user=User(first_name='Triển đẹp trai', id=6243409373, is_bot=True, username='triendt1_bot'), group_chat_created=False, message_id=102, supergroup_chat_created=False)